In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas
import datetime
import os
import sys
module_path = os.path.abspath(os.path.join('../data_pipeline/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import sqlite3
import create_onset_data

In [2]:
inds_dbdir = "/home/bharat/Documents/data/ss_onset_dataset/data/sqlite3/"
sm_db_name = "smu_sml_sme.sqlite"
sm_table_name = "smusmlsme"

In [3]:
dataObj = create_onset_data.OnsetData(delTCutoff=2)
ssBinDF = dataObj.create_output_bins(\
                saveFile="../data/binned_data_extra_latlon.feather",\
                aulDBdir=inds_dbdir, \
                 aulDBName=sm_db_name,\
                 aulTabName=sm_table_name)
# print ssBinDF.head()
# print "-------------"
# print ssBinDF.shape

('data distribution---->', {0: 14482, 1: 19059, 2: 19047, 3: 6347})
('more data found than required---', 4577, 28304)
no common rows found between ss and non-ss data
('expanded with new non-SS data--->', 58935, 63512)
                     bin_0  bin_1    mlat_0  mlat_1     mlt_0  mlt_1  \
1996-03-30 03:41:00      0      0 -1.000000    -1.0 -1.000000   -1.0   
1996-03-30 03:42:00      1      0  0.763333    -1.0  0.998056   -1.0   
1996-03-30 03:43:00      1      0  0.763333    -1.0  0.998056   -1.0   
1996-03-30 03:44:00      1      0  0.763333    -1.0  0.998056   -1.0   
1996-03-30 03:45:00      1      0  0.763333    -1.0  0.998056   -1.0   

                     del_minutes data_label  
1996-03-30 03:41:00        30.95          P  
1996-03-30 03:42:00        29.95          P  
1996-03-30 03:43:00        28.95          P  
1996-03-30 03:44:00        27.95          P  
1996-03-30 03:45:00        26.95          P  
